## Importing Necessary Libraries

In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd 


In [2]:
# Importing Libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

## READING DATA SET

In [3]:
# Read the collected data into pandas DataFrames.
data = pd.read_csv('Dataset_Uber Traffic.csv')
# Check the head of the dataset
data.head()

,DateTime,Junction,Vehicles,ID
0,01/11/15 0:00,1,15,20151101001
1,01/11/15 1:00,1,13,20151101011
2,01/11/15 2:00,1,10,20151101021
3,01/11/15 3:00,1,7,20151101031
4,01/11/15 4:00,1,9,20151101041


Inspect the various aspects of the Uber dataframe

In [4]:
data.shape

(48120, 4)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48120 entries, 0 to 48119
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DateTime  48120 non-null  object
 1   Junction  48120 non-null  int64 
 2   Vehicles  48120 non-null  int64 
 3   ID        48120 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [6]:
data.describe()

,Junction,Vehicles,ID
count,48120.000000,48120.000000,4.812000e+04
mean,2.180549,22.791334,2.016330e+10
std,0.966955,20.750063,5.944854e+06
min,1.000000,1.000000,2.015110e+10
25%,1.000000,9.000000,2.016042e+10
50%,2.000000,15.000000,2.016093e+10
75%,3.000000,29.000000,2.017023e+10
max,4.000000,180.000000,2.017063e+10


Let's do a sanity check on the dataframe for missing values.

## Data Cleaning

In [7]:
# percentage of missing values in each column
round(data.isnull().sum()/len(data.index), 2)*100

DateTime    0.0
Junction    0.0
Vehicles    0.0
ID          0.0
dtype: float64

In [8]:
# Loking for duplicate values,if any
#Checking for duplicate rows
duplicates = data.duplicated()
print(f'Number of duplicate rows: {duplicates.sum()}')

Number of duplicate rows: 0


In the above dataframe there are no duplicate rows

In [9]:
# Ensuring ift he columns have appropriate data types
print(data.dtypes)

DateTime    object
Junction     int64
Vehicles     int64
ID           int64
dtype: object


In [10]:
#Type conversion
# Converting 'DateTime' column to datetime type
data['DateTime'] = pd.to_datetime(data['DateTime'])

In [11]:
# Handling outliers using IQR method

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = data['Vehicles'].quantile(0.25)
Q3 = data['Vehicles'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier thresholds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
outliers = data[(data['Vehicles'] < lower_bound) | (data['Vehicles'] > upper_bound)]
print(f'Number of outliers: {outliers.shape[0]}')

Number of outliers: 3617


In [12]:
#Removing Outliers
data_clean = data[(data['Vehicles'] >= lower_bound) & (data['Vehicles'] <= upper_bound)]

In [13]:
# Verify if outliers are removed
print(f'Original data size: {data.shape[0]}')
print(f'Cleaned data size: {data_clean.shape[0]}')

Original data size: 48120
Cleaned data size: 44503


In [14]:
data["DateTime"] = pd.to_datetime(data["DateTime"])
traffic_summary = (
    data.groupby(["Junction", pd.Grouper(key="DateTime", freq="H")])["Vehicles"]
    .sum()
    .reset_index()
)

No outliers detected

In [15]:
# Checking the range of dates
print(f'Start Date: {data["DateTime"].min()}')
print(f'End Date: {data["DateTime"].max()}')

# Check for consistency in 'Junction' and 'ID' columns
print(data['Junction'].unique())
print(data['ID'].unique())

Start Date: 2015-01-11 00:00:00
End Date: 2017-12-06 23:00:00
[1 2 3 4]
[20151101001 20151101011 20151101021 ... 20170630214 20170630224
 20170630234]


Data is consistent and covers a time span from January 11, 2015, to December 6, 2017

## Aggregate traffic data

#### - Compile traffic data into hourly intervals for each junction.
#### - Ensure data includes relevant details such as vehicle counts.

Resampling the data into hourly intervals with vehicle count 

In [16]:
# Seting 'DateTime' as the index for resampling
data.set_index('DateTime', inplace=True)

# Resampling the data to hourly intervals and aggregate vehicle counts
df_hourly = data.groupby('Junction').resample('H').agg({'Vehicles': 'sum'}).reset_index()

# Check the first few rows of the aggregated data
print(df_hourly.head())

   Junction            DateTime  Vehicles
0         1 2015-01-11 00:00:00        15
1         1 2015-01-11 01:00:00        13
2         1 2015-01-11 02:00:00        10
3         1 2015-01-11 03:00:00         7
4         1 2015-01-11 04:00:00         9


Let's verify if there are any missing hours and data for a specific junction

In [17]:
# Check for missing hours
missing_hours = df_hourly[df_hourly['Vehicles'].isna()]
print(f'Missing hours: {missing_hours.shape[0]}')

Missing hours: 0


In [18]:
# Checking the data for a specific junction
junction_id = 4  # Change this to the junction of interest
df_junction_hourly = df_hourly[df_hourly['Junction'] == junction_id]
print(df_junction_hourly.head())

       Junction            DateTime  Vehicles
76392         4 2017-01-01 00:00:00         3
76393         4 2017-01-01 01:00:00         1
76394         4 2017-01-01 02:00:00         4
76395         4 2017-01-01 03:00:00         4
76396         4 2017-01-01 04:00:00         2


### Normalization

In [19]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

#Extracting HourOfDay
df_hourly['HourOfDay'] = df_hourly['DateTime'].dt.hour

# Define features
features = ['Vehicles', 'Junction', 'HourOfDay']

# Separate numeric and categorical features
numeric_features = ['Vehicles', 'HourOfDay']
categorical_features = ['Junction']

# Normalize numeric features
scaler = MinMaxScaler()
df_hourly[numeric_features] = scaler.fit_transform(df_hourly[numeric_features])

# Encode categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_categorical = encoder.fit_transform(df_hourly[categorical_features])
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_features))

# Concatenate normalized numeric and encoded categorical features
df_processed = pd.concat([df_hourly[numeric_features], encoded_categorical_df], axis=1)

# Check the processed data
print(df_processed.head())

   Vehicles  HourOfDay  Junction_1  Junction_2  Junction_3  Junction_4
0  0.083333   0.000000         1.0         0.0         0.0         0.0
1  0.072222   0.043478         1.0         0.0         0.0         0.0
2  0.055556   0.086957         1.0         0.0         0.0         0.0
3  0.038889   0.130435         1.0         0.0         0.0         0.0
4  0.050000   0.173913         1.0         0.0         0.0         0.0


### Data is processed for comprisons accross different timeperiods and junctions